In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

C:\Users\tymek\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\tymek\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [7]:
class MLP:
    def __init__(self, input_size, hidden_layer_sizes, output_size, activations=None, weights=None, biases=None):
        if activations is None:
            self.activations = ["sigmoid"] * (len(hidden_layer_sizes)) + ["linear"]
        else:
            self.activations = activations
        if weights is None:
            self.weights = [np.random.randn(input_size, hidden_layer_sizes[0])]
            self.weights += [np.random.randn(hidden_layer_sizes[i], hidden_layer_sizes[i+1]) for i in range(len(hidden_layer_sizes)-1)]
            self.weights += [(np.random.randn(hidden_layer_sizes[-1], output_size))]
        else:
            self.weights = [np.array(layer) for layer in weights]
        if biases is None:
            self.biases = [np.random.randn(size) for size in hidden_layer_sizes]
            self.biases+= [(np.random.randn(output_size))]
        else:
            self.biases = biases
            
        self.activation_functions = {
            "sigmoid": lambda x: 1 / (1 + np.exp(-x)),
            "linear": lambda x: x,
            "relu": lambda x: np.maximum(0, x)
        }
            
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
            

    def forward(self, x):
        activations = [x]
        for i in range(len(self.weights)):
            x = self.activation_functions[self.activations[i]](np.dot(x, self.weights[i]) + self.biases[i])
            activations.append(x)
        return activations
        
    def predict(self, X):
        predictions = []
        for x in X:
            predictions.append(self.forward(x)[-1])
        return np.array(predictions)
        
    def mse(self, X, y):
        predictions = self.predict(X)
        return np.mean((predictions - y) ** 2)

    def backward(self, X, y, learning_rate):
        activations = self.forward(X)
        deltas = [None] * len(self.weights)
        deltas[-1] = activations[-1] - y.reshape(-1, 1)
        for i in reversed(range(len(deltas) - 1)):
            if self.activations[i] == "sigmoid":
                deltas[i] = np.dot(deltas[i+1], self.weights[i+1].T) * (activations[i+1] * (1 - activations[i+1]))
            elif self.activations[i] == "linear":
                deltas[i] = np.dot(deltas[i+1], self.weights[i+1].T)
            elif self.activations[i] == "relu":
                deltas[i] = np.dot(deltas[i+1], self.weights[i+1].T) * (activations[i+1] > 0)

        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * np.dot(activations[i].T, deltas[i])
            self.biases[i] -= learning_rate * np.mean(deltas[i], axis=0)

    def train(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            self.backward(X, y, learning_rate)
            if epoch % 100 == 0:
                loss = self.mse(X, y)
                print(f"Epoch {epoch}: Loss = {loss}")

In [3]:
square_simple_df = pd.read_csv("dane/regression/square-simple-test.csv", index_col=0)
steps_large_df = pd.read_csv("dane/regression/steps-large-test.csv", index_col=0)

In [14]:
weights = [np.array([[ 54.382896 , -55.09526  ,  14.7870655,  27.029877 ,  55.166576 ,
         59.808445 , -27.606133 ,  59.795853 , -54.91213  ,  54.853874 ]]), np.array([[ 12.019116],
       [-33.381557],
       [-90.85054 ],
       [ 96.54781 ],
       [ 19.363287],
       [ 55.674763],
       [-71.23986 ],
       [ 55.58958 ],
       [ 31.410458],
       [ 16.47028 ]])]
biases = [np.array([-27.033913,  27.38708 , -22.158144, -40.350998, -27.422424,
        29.846415,  41.21274 ,  29.840431, -29.45995 , -27.267405]), np.array([-6.6802287])]

In [17]:
model = MLP(input_size=1, hidden_layer_sizes=[10], output_size=1, weights=weights, biases=biases)

In [18]:
model.train(steps_large_df.x.to_numpy().reshape(-1, 1), steps_large_df.y.to_numpy().reshape(-1, 1), learning_rate=0.00001, epochs=10000000)

Epoch 0: Loss = 34.05578498090655
Epoch 100: Loss = 34.03369465099391
Epoch 200: Loss = 34.030905147678574
Epoch 300: Loss = 34.02978123316768
Epoch 400: Loss = 34.02911029511515
Epoch 500: Loss = 34.02860802976399
Epoch 600: Loss = 34.02818568522004
Epoch 700: Loss = 34.02780881513113
Epoch 800: Loss = 34.0274608567411
Epoch 900: Loss = 34.02713233432382
Epoch 1000: Loss = 34.02681722082388
Epoch 1100: Loss = 34.02651147507752
Epoch 1200: Loss = 34.02621232220088
Epoch 1300: Loss = 34.02591783659187
Epoch 1400: Loss = 34.02562667538838
Epoch 1500: Loss = 34.0253378996486
Epoch 1600: Loss = 34.02505085173189
Epoch 1700: Loss = 34.024765070392895
Epoch 1800: Loss = 34.024480231719615
Epoch 1900: Loss = 34.024196107949
Epoch 2000: Loss = 34.023912538705446
Epoch 2100: Loss = 34.0236294108916
Epoch 2200: Loss = 34.02334664461533
Epoch 2300: Loss = 34.02306418333155
Epoch 2400: Loss = 34.022781986930774
Epoch 2500: Loss = 34.02250002688813
Epoch 2600: Loss = 34.02221828285657
Epoch 2700: L

Epoch 22000: Loss = 33.970450559202504
Epoch 22100: Loss = 33.97019729468232
Epoch 22200: Loss = 33.96994415806943
Epoch 22300: Loss = 33.969691149201665
Epoch 22400: Loss = 33.96943826791722
Epoch 22500: Loss = 33.969185514054466
Epoch 22600: Loss = 33.96893288745198
Epoch 22700: Loss = 33.96868038794859
Epoch 22800: Loss = 33.968428015383296
Epoch 22900: Loss = 33.96817576959537
Epoch 23000: Loss = 33.96792365042429
Epoch 23100: Loss = 33.96767165770983
Epoch 23200: Loss = 33.967419791291775
Epoch 23300: Loss = 33.96716805101035
Epoch 23400: Loss = 33.966916436705844
Epoch 23500: Loss = 33.96666494821889
Epoch 23600: Loss = 33.96641358539027
Epoch 23700: Loss = 33.966162348060934
Epoch 23800: Loss = 33.96591123607228
Epoch 23900: Loss = 33.96566024926553
Epoch 24000: Loss = 33.96540938748249
Epoch 24100: Loss = 33.96515865056505
Epoch 24200: Loss = 33.96490803835524
Epoch 24300: Loss = 33.96465755069538
Epoch 24400: Loss = 33.96440718742806
Epoch 24500: Loss = 33.964156948395974
Epoc

Epoch 43500: Loss = 33.91869463176346
Epoch 43600: Loss = 33.9184654764556
Epoch 43700: Loss = 33.918236418849396
Epoch 43800: Loss = 33.918007458824036
Epoch 43900: Loss = 33.91777859625884
Epoch 44000: Loss = 33.917549831033334
Epoch 44100: Loss = 33.91732116302725
Epoch 44200: Loss = 33.91709259212045
Epoch 44300: Loss = 33.91686411819284
Epoch 44400: Loss = 33.916635741124765
Epoch 44500: Loss = 33.916407460796414
Epoch 44600: Loss = 33.91617927708843
Epoch 44700: Loss = 33.91595118988135
Epoch 44800: Loss = 33.915723199056124
Epoch 44900: Loss = 33.91549530449364
Epoch 45000: Loss = 33.91526750607512
Epoch 45100: Loss = 33.91503980368187
Epoch 45200: Loss = 33.91481219719536
Epoch 45300: Loss = 33.914584686497214
Epoch 45400: Loss = 33.914357271469235
Epoch 45500: Loss = 33.91412995199337
Epoch 45600: Loss = 33.91390272795171
Epoch 45700: Loss = 33.91367559922664
Epoch 45800: Loss = 33.913448565700484
Epoch 45900: Loss = 33.913221627255936
Epoch 46000: Loss = 33.91299478377564
Epo

Epoch 65200: Loss = 33.87106868290825
Epoch 65300: Loss = 33.87085815399502
Epoch 65400: Loss = 33.87064770005796
Epoch 65500: Loss = 33.870437321007046
Epoch 65600: Loss = 33.870227016752295
Epoch 65700: Loss = 33.870016787203895
Epoch 65800: Loss = 33.86980663227219
Epoch 65900: Loss = 33.869596551867566
Epoch 66000: Loss = 33.869386545900554
Epoch 66100: Loss = 33.8691766142818
Epoch 66200: Loss = 33.86896675692212
Epoch 66300: Loss = 33.868756973732424
Epoch 66400: Loss = 33.86854726462369
Epoch 66500: Loss = 33.86833762950711
Epoch 66600: Loss = 33.86812806829389
Epoch 66700: Loss = 33.867918580895456
Epoch 66800: Loss = 33.86770916722324
Epoch 66900: Loss = 33.86749982718892
Epoch 67000: Loss = 33.86729056070422
Epoch 67100: Loss = 33.86708136768096
Epoch 67200: Loss = 33.86687224803115
Epoch 67300: Loss = 33.866663201666846
Epoch 67400: Loss = 33.86645422850034
Epoch 67500: Loss = 33.86624532844384
Epoch 67600: Loss = 33.866036501409866
Epoch 67700: Loss = 33.865827747310966
Epo

Epoch 86800: Loss = 33.82719628774972
Epoch 86900: Loss = 33.827000048973815
Epoch 87000: Loss = 33.8268038682726
Epoch 87100: Loss = 33.82660774557863
Epoch 87200: Loss = 33.82641168082456
Epoch 87300: Loss = 33.826215673943096
Epoch 87400: Loss = 33.82601972486703
Epoch 87500: Loss = 33.8258238335293
Epoch 87600: Loss = 33.82562799986284
Epoch 87700: Loss = 33.82543222380081
Epoch 87800: Loss = 33.825236505276244
Epoch 87900: Loss = 33.825040844222485
Epoch 88000: Loss = 33.82484524057291
Epoch 88100: Loss = 33.8246496942609
Epoch 88200: Loss = 33.82445420521995
Epoch 88300: Loss = 33.8242587733837
Epoch 88400: Loss = 33.824063398685865
Epoch 88500: Loss = 33.823868081060226
Epoch 88600: Loss = 33.82367282044061
Epoch 88700: Loss = 33.82347761676099
Epoch 88800: Loss = 33.82328246995546
Epoch 88900: Loss = 33.82308737995809
Epoch 89000: Loss = 33.82289234670317
Epoch 89100: Loss = 33.82269737012498
Epoch 89200: Loss = 33.82250245015789
Epoch 89300: Loss = 33.82230758673642
Epoch 8940

...

Epoch 1927800: Loss = 28.327788593749283
Epoch 1927900: Loss = 28.32767830498543
Epoch 1928000: Loss = 28.3275680220313
Epoch 1928100: Loss = 28.32745774488543
Epoch 1928200: Loss = 28.327347473546222
Epoch 1928300: Loss = 28.327237208012207
Epoch 1928400: Loss = 28.327126948281858
Epoch 1928500: Loss = 28.32701669435369
Epoch 1928600: Loss = 28.326906446226108
Epoch 1928700: Loss = 28.32679620389765
Epoch 1928800: Loss = 28.326685967366775
Epoch 1928900: Loss = 28.326575736631955
Epoch 1929000: Loss = 28.326465511691758
Epoch 1929100: Loss = 28.326355292544527
Epoch 1929200: Loss = 28.326245079188848
Epoch 1929300: Loss = 28.32613487162315
Epoch 1929400: Loss = 28.326024669845957
Epoch 1929500: Loss = 28.325914473855825
Epoch 1929600: Loss = 28.32580428365103
Epoch 1929700: Loss = 28.325694099230287
Epoch 1929800: Loss = 28.325583920591956
Epoch 1929900: Loss = 28.32547374773455
Epoch 1930000: Loss = 28.325363580656617
Epoch 1930100: Loss = 28.32525341935653
Epoch 1930200: Loss = 28.3

Epoch 1948200: Loss = 28.305407903052128
Epoch 1948300: Loss = 28.305298768803326
Epoch 1948400: Loss = 28.305189640066153
Epoch 1948500: Loss = 28.305080516839215
Epoch 1948600: Loss = 28.304971399121168
Epoch 1948700: Loss = 28.304862286910463
Epoch 1948800: Loss = 28.304753180205896
Epoch 1948900: Loss = 28.304644079005943
Epoch 1949000: Loss = 28.30453498330924
Epoch 1949100: Loss = 28.30442589311436
Epoch 1949200: Loss = 28.304316808419898
Epoch 1949300: Loss = 28.304207729224494
Epoch 1949400: Loss = 28.30409865552678
Epoch 1949500: Loss = 28.303989587325297
Epoch 1949600: Loss = 28.303880524618645
Epoch 1949700: Loss = 28.303771467405497
Epoch 1949800: Loss = 28.303662415684364
Epoch 1949900: Loss = 28.303553369453933
Epoch 1950000: Loss = 28.30344432871277
Epoch 1950100: Loss = 28.303335293459504
Epoch 1950200: Loss = 28.303226263692732
Epoch 1950300: Loss = 28.303117239411076
Epoch 1950400: Loss = 28.30300822061309
Epoch 1950500: Loss = 28.302899207297397
Epoch 1950600: Loss =

Epoch 1968600: Loss = 28.283256723818983
Epoch 1968700: Loss = 28.283148685670493
Epoch 1968800: Loss = 28.28304065275873
Epoch 1968900: Loss = 28.282932625082424
Epoch 1969000: Loss = 28.282824602640314
Epoch 1969100: Loss = 28.282716585431068
Epoch 1969200: Loss = 28.28260857345339
Epoch 1969300: Loss = 28.282500566706037
Epoch 1969400: Loss = 28.282392565187635
Epoch 1969500: Loss = 28.282284568896944
Epoch 1969600: Loss = 28.282176577832725
Epoch 1969700: Loss = 28.282068591993593
Epoch 1969800: Loss = 28.281960611378278
Epoch 1969900: Loss = 28.281852635985572
Epoch 1970000: Loss = 28.28174466581409
Epoch 1970100: Loss = 28.281636700862563
Epoch 1970200: Loss = 28.281528741129712
Epoch 1970300: Loss = 28.28142078661427
Epoch 1970400: Loss = 28.28131283731491
Epoch 1970500: Loss = 28.281204893230452
Epoch 1970600: Loss = 28.281096954359423
Epoch 1970700: Loss = 28.280989020700712
Epoch 1970800: Loss = 28.28088109225293
Epoch 1970900: Loss = 28.28077316901487
Epoch 1971000: Loss = 2

Epoch 1988900: Loss = 28.261430604715272
Epoch 1989000: Loss = 28.261323603763742
Epoch 1989100: Loss = 28.261216607796175
Epoch 1989200: Loss = 28.261109616811407
Epoch 1989300: Loss = 28.261002630808136
Epoch 1989400: Loss = 28.26089564978525
Epoch 1989500: Loss = 28.26078867374157
Epoch 1989600: Loss = 28.260681702675875
Epoch 1989700: Loss = 28.260574736586918
Epoch 1989800: Loss = 28.260467775473582
Epoch 1989900: Loss = 28.260360819334654
Epoch 1990000: Loss = 28.26025386816889
Epoch 1990100: Loss = 28.26014692197516
Epoch 1990200: Loss = 28.26003998075223
Epoch 1990300: Loss = 28.259933044498936
Epoch 1990400: Loss = 28.259826113214032
Epoch 1990500: Loss = 28.25971918689643
Epoch 1990600: Loss = 28.259612265544796
Epoch 1990700: Loss = 28.259505349158033
Epoch 1990800: Loss = 28.259398437734983
Epoch 1990900: Loss = 28.259291531274393
Epoch 1991000: Loss = 28.25918462977507
Epoch 1991100: Loss = 28.259077733235873
Epoch 1991200: Loss = 28.258970841655557
Epoch 1991300: Loss = 2

KeyboardInterrupt: 

In [21]:
print(model.weights)
print(model.biases)

[array([[ 59.84945643, -60.38693972,  16.41401521,  28.18702789,
         58.81308855,  61.7699252 , -28.60149239,  61.75053536,
        -56.93791512,  57.97013632]]), array([[ -69.99614201],
       [ -69.63517256],
       [-110.31905786],
       [ 102.8609668 ],
       [  47.2780337 ],
       [  80.68766256],
       [ -84.33800145],
       [  80.47651883],
       [  80.77069351],
       [  33.81193926]])]
[array([-27.42018337,  29.25488331, -24.57718061, -42.08837979,
       -28.4247261 ,  31.06034746,  42.70865137,  31.05145369,
       -29.68925833, -27.97675392]), array([-6.67432156])]


In [20]:
model.biases

[array([-27.42018337,  29.25488331, -24.57718061, -42.08837979,
        -28.4247261 ,  31.06034746,  42.70865137,  31.05145369,
        -29.68925833, -27.97675392]),
 array([-6.67432156])]

In [ ]:
def MSE(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)